In [1]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
REGION = 'us-central1'
BUCKET_URI = 'gs://kubeflow-mlops-410520-cloudbuild-pipelines'

## Create Dockerfile

In [2]:
%%writefile kfp-cli/Dockerfile


FROM gcr.io/deeplearning-platform-release/base-cpu
WORKDIR /kfp-cli
ADD pipeline.py ./pipeline.py
ADD requirements.txt ./requirements.txt
RUN pip install -r requirements.txt
CMD python ./pipeline.py


Overwriting kfp-cli/Dockerfile


## create a bucket to house our pipeline yaml 

In [4]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://kubeflow-mlops-410520-cloudbuild-pipelines/...


In [4]:
IMAGE_NAME='kfp-truiz-mlops-v2'
TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [ ]:
! docker build -t my-image:latest .  

In [5]:
!gcloud builds submit --timeout 15m --tag {IMAGE_URI} kfp-cli


Creating temporary tarball archive of 6 file(s) totalling 12.6 KiB before compression.
Uploading tarball of [kfp-cli] to [gs://kubeflow-mlops-410520_cloudbuild/source/1713401559.251934-5d75d9e3da1741ce9c5aecc0cce7b883.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/kubeflow-mlops-410520/locations/global/builds/8f38fca5-26e6-48e5-bf67-8b94a3fd5aab].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/8f38fca5-26e6-48e5-bf67-8b94a3fd5aab?project=990864364836 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "8f38fca5-26e6-48e5-bf67-8b94a3fd5aab"

FETCHSOURCE
Fetching storage object: gs://kubeflow-mlops-410520_cloudbuild/source/1713401559.251934-5d75d9e3da1741ce9c5aecc0cce7b883.tgz#1713401559556908
Copying gs://kubeflow-mlops-410520_cloudbuild/source/1713401559.251934-5d75d9e3da1741ce9c5aecc0cce7b883.tgz#1713401559556908...
/ [1 files][  2.8 KiB/  2.8 KiB]                                                
O

In [6]:
!gcloud builds submit ./kfp-cli --config cloudbuild.yaml

Creating temporary tarball archive of 6 file(s) totalling 12.8 KiB before compression.
Uploading tarball of [./kfp-cli] to [gs://kubeflow-mlops-410520_cloudbuild/source/1712691159.633853-751f6828b16f4826aa66e873f271297f.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/kubeflow-mlops-410520/locations/global/builds/662f9c1a-4a5f-497a-8f7e-daeb44cf0e4e].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/662f9c1a-4a5f-497a-8f7e-daeb44cf0e4e?project=990864364836 ].

gcloud builds submit only displays logs from Cloud Storage. To view logs from Cloud Logging, run:
gcloud beta builds submit

ID                                    CREATE_TIME                DURATION  SOURCE                                                                                               IMAGES  STATUS
662f9c1a-4a5f-497a-8f7e-daeb44cf0e4e  2024-04-09T19:32:40+00:00  3M37S     gs://kubeflow-mlops-410520_cloudbuild/source/1712691159.633853-751f6828b16f4826aa66e873f271297f.tgz  -    